<div style="background-color: #fff3cd; border: 1px solid #ffeeba; border-radius: 5px; padding: 15px;">
    <h3 style="color: #856404; margin-top: 0;">⚠️ Demo Artifact Only: Consumption Layer</h3>
    <p style="color: #856404; margin-bottom: 0;">
        This notebook is designed for <strong>Interactive Consumption & Verification</strong> of the pipeline results.
    </p>
    <ul style="color: #856404;">
        <li>It reads strictly from the <strong>Gold Layer</strong> (Customer 360 & GenAI Campaigns).</li>
        <li>It requires <strong>User Interaction</strong> (Dropdown Widgets) to function.</li>
        <li><strong>NOTE:</strong> This notebook is <u>NOT</u> part of the automated Databricks Job/Workflow.</li>
    </ul>
</div>

### Setup & Data Loading
We fetch the Top 20 relevant customers and bring them into memory (Pandas) so the interactive widgets respond instantly without running Spark jobs.

In [0]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

# Set Context
spark.sql("USE CATALOG olist_hackathon")

print("⏳ Loading CRM Data...")

# 1. Join Profile (Gold) + GenAI Content (Campaigns)
# We need the Churn Status from 360 and the Email from Campaigns
df_crm = spark.sql("""
    SELECT 
        a.customer_unique_id,
        a.favorite_category,
        a.monetary_value,
        a.avg_review_score,
        a.frequency,
        b.generated_email
    FROM gold.customer_360 a
    JOIN gold.retention_campaigns b 
      ON a.customer_unique_id = b.customer_unique_id
    WHERE a.is_churn_risk = 1
    ORDER BY a.monetary_value DESC
    LIMIT 20
""").toPandas()

print(f"✅ Loaded {len(df_crm)} High-Value Profiles for the Interactive Demo.")

### The Interactive CRM App 
This code builds the UI using ipywidgets. It dynamically renders HTML cards based on your selection.

In [0]:
# --- Helper: Star Rating Generator ---
def get_stars(score):
    full_stars = int(score)
    empty_stars = 5 - full_stars
    return "⭐" * full_stars + "☆" * empty_stars

# --- Helper: HTML Renderer ---
def render_profile(customer_id):
    # Fetch row
    row = df_crm[df_crm['customer_unique_id'] == customer_id].iloc[0]
    
    # Format Data
    money = f"${row['monetary_value']:,.2f}"
    stars = get_stars(row['avg_review_score'])
    category = row['favorite_category'].replace('_', ' ').title()
    email_text = row['generated_email'].replace('\n', '<br>')
    
    # Dynamic HTML Template
    html = f"""
    <div style="font-family: sans-serif; border: 1px solid #ddd; border-radius: 10px; padding: 20px; max-width: 800px; box-shadow: 2px 2px 10px rgba(0,0,0,0.1);">
        
        <div style="display: flex; justify-content: space-between; align-items: center; border-bottom: 1px solid #eee; padding-bottom: 10px; margin-bottom: 15px;">
            <div>
                <h2 style="margin: 0; color: #333;">Customer Profile</h2>
                <small style="color: #777;">ID: {row['customer_unique_id']}</small>
            </div>
            <div style="background-color: #ffebee; color: #c62828; padding: 5px 15px; border-radius: 20px; font-weight: bold; font-size: 14px; border: 1px solid #ffcdd2;">
                ⚠️ High Churn Risk
            </div>
        </div>

        <div style="display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 15px; margin-bottom: 20px;">
            <div style="background: #f9f9f9; padding: 10px; border-radius: 8px; text-align: center;">
                <div style="font-size: 12px; color: #555;">Total Lifetime Value</div>
                <div style="font-size: 20px; font-weight: bold; color: #2e7d32;">{money}</div>
            </div>
            <div style="background: #f9f9f9; padding: 10px; border-radius: 8px; text-align: center;">
                <div style="font-size: 12px; color: #555;">Top Category</div>
                <div style="font-size: 18px; font-weight: bold; color: #0277bd;">{category}</div>
            </div>
             <div style="background: #f9f9f9; padding: 10px; border-radius: 8px; text-align: center;">
                <div style="font-size: 12px; color: #555;">Satisfaction</div>
                <div style="font-size: 18px; color: #f9a825;">{stars}</div>
                <div style="font-size: 10px;">({row['avg_review_score']} / 5.0)</div>
            </div>
        </div>

        <h3 style="margin-bottom: 10px; color: #444;">✨ GenAI Retention Action (Llama 3)</h3>
        <div style="background-color: #f0f4f8; border-left: 5px solid #0467DF; padding: 15px; border-radius: 5px; font-family: monospace; font-size: 14px; color: #333; line-height: 1.5;">
            {email_text}
        </div>
        <div style="margin-top: 5px; font-size: 11px; color: #666; font-style: italic;">
            * This content was auto-generated based on the customer's sentiment and purchase history.
        </div>
    </div>
    """
    display(HTML(html))

# --- Widget Logic ---
dropdown = widgets.Dropdown(
    options=df_crm['customer_unique_id'].tolist(),
    description='Select User:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

output = widgets.Output()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            output.clear_output()
            render_profile(change['new'])

dropdown.observe(on_change)

# Initialize with the first user
with output:
    render_profile(df_crm['customer_unique_id'].iloc[0])

# Display App
display(widgets.VBox([dropdown, output]))